# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1260


In [2]:
using CuArrays

## 讀取資料

In [3]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [4]:
println(size(train_X))
println(size(train_y))
println(size(test_X))
println(size(test_y))

(32, 32, 3, 20000)
(20000,)
(32, 32, 3, 2000)
(2000,)


In [5]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

## hyper parameters

In [6]:
epochs = 20;
batchsize = 64;
learining_rate = 0.002;
decay = 0.1;
decay_step = 1;
clip = 1e-4;
optimizer = Flux.Optimiser(ExpDecay(learining_rate, decay, decay_step, clip), ADAM(learining_rate));

## loss function

In [8]:
loss(x, y) = logitcrossentropy(model(x), y);

## callback function

In [9]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

evalcb() = @show(test_loss());

## evaluation function

In [10]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

## CNN 模型
Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)

In [11]:
model = Chain(
    Conv((3, 3), 3=>16, pad=(1,1), relu),
    MaxPool((2, 2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2, 2)),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2, 2)),
    flatten,
    Dense(512, 10),
    logsoftmax)

Chain(Conv((3, 3), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(512, 10), logsoftmax)

## using CUDA

In [12]:
model = model |> gpu;
train_X = train_X |> gpu;
train_y = train_y |> gpu;
test_X = test_X |> gpu;
test_y = test_y |> gpu;

## data loader

In [15]:
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true);
test = DataLoader(test_X, test_y, batchsize=batchsize);

## training model

In [16]:
@epochs epochs Flux.train!(loss, params(model), train, optimizer, cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.287951f0


┌ Info: Epoch 2
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.1506466f0


┌ Info: Epoch 3
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 0.7334339f0
test_loss() = 0.7139645f0


┌ Info: Epoch 18
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 0.7340827f0


┌ Info: Epoch 19
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 20
└ @ Main C:\Users\kai\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 0.79189324f0


## model evaluation

In [17]:
using Printf
@printf("Accuracy: %.2f%%\n", accuracy(test_X, test_y)*100)

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays C:\Users\kai\.julia\packages\GPUArrays\OXvxB\src\host\indexing.jl:43


Accuracy: 81.25%
